In [1]:
# import libraries
import numpy as np
np.set_printoptions(precision = 4, suppress = True)
import pandas as pd
import os
from datetime import datetime
%matplotlib qt

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# import modules
from widgets import CandlestickWidget

In [2]:
out_data_dir = "data/out"
n_levels = 4

In [3]:
def get_data_points(csv_path, sep=",", date_col="time"):    
    # read file
    data_points = pd.read_csv(csv_path, sep)
    
    # convert time
    data_points[date_col] = data_points[date_col].apply(lambda x: datetime.utcfromtimestamp(x))
    
    # set index
    data_points.set_index(date_col, inplace=True)

    return data_points

In [5]:
candlestick_points = get_data_points(os.path.join(out_data_dir, "candles.csv"))
open_points = get_data_points(os.path.join(out_data_dir, "open_points.csv"))
close_points = get_data_points(os.path.join(out_data_dir, "close_points.csv"))
mean_price_points = get_data_points(os.path.join(out_data_dir, "mean_points.csv"))
indic_points = get_data_points(os.path.join(out_data_dir, "indic_points.csv"))

In [6]:
def group_points(points):
    return points.groupby(points.index).agg(lambda x: list(x))

In [7]:
open_points = group_points(open_points)

In [8]:
prices_levels = np.full((len(open_points.index), n_levels), np.nan)

for row_idx, row in enumerate(open_points.iterrows()):
    prices = row[1]["price"]
    
    for col_idx, price in enumerate(prices):
        prices_levels[row_idx][col_idx] = price

In [9]:
open_points_levels = []

for col_idx in range(n_levels):
    open_points_levels.append(pd.DataFrame(data=prices_levels[:, col_idx], index=open_points.index,
                                           columns=open_points.columns))

In [10]:
# reindex
close_points = close_points.reindex(candlestick_points.index)
mean_price_points = mean_price_points.reindex(candlestick_points.index)
indic_points = indic_points.reindex(candlestick_points.index)

for col_idx in range(n_levels):
    open_points_levels[col_idx] = open_points_levels[col_idx].reindex(candlestick_points.index)

In [11]:
pair = "BTC/USDT"
interval = "1 hour"
title = f"{pair}\n{interval}"

In [12]:
candles = CandlestickWidget(candlestick_points, indic_points, mean_price_points, open_points_levels, close_points, title)

In [13]:
candles.show()